# Doc2Vec to wikipedia articles

We conduct the replication to **Document Embedding with Paragraph Vectors** (http://arxiv.org/abs/1507.07998).
In this paper, they showed only DBOW results to Wikipedia data. So we replicate this experiments using not only DBOW but also DM.

## Basic Setup

In [1]:
! pip3 install gensim

  Using cached numpy-1.12.1-cp35-cp35m-manylinux1_x86_64.whl
  Using cached six-1.10.0-py2.py3-none-any.whl
  Using cached scipy-0.19.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached boto-2.47.0-py2.py3-none-any.whl
  Using cached requests-2.17.0-py2.py3-none-any.whl
  Using cached chardet-3.0.3-py2.py3-none-any.whl
  Using cached certifi-2017.4.17-py2.py3-none-any.whl
  Using cached idna-2.5-py2.py3-none-any.whl
  Using cached urllib3-1.21.1-py2.py3-none-any.whl
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# First Always set logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)  

Let's import Doc2Vec module.

In [3]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing

2017-05-29 19:39:04,614 : INFO : 'pattern' package not found; tag filters are not available for English


## Preparing the corpus

First, download the dump of all Wikipedia articles from [here](http://download.wikimedia.org/enwiki/) (you want the file enwiki-latest-pages-articles.xml.bz2, or enwiki-YYYYMMDD-pages-articles.xml.bz2 for date-specific dumps).

Second, convert the articles to WikiCorpus. WikiCorpus construct a corpus from a Wikipedia (or other MediaWiki-based) database dump.

For more details on WikiCorpus, you should access [Corpus from a Wikipedia dump](https://radimrehurek.com/gensim/corpora/wikicorpus.html).

In [1]:
print('Start creating wiki corpus:')
wiki = WikiCorpus("data/enwiki-20170520-pages-articles.xml.bz2")
print('Finished')
#wiki = WikiCorpus("enwiki-YYYYMMDD-pages-articles.xml.bz2")

SyntaxError: invalid syntax (<ipython-input-1-f331338be95a>, line 2)

Define **TaggedWikiDocument** class to convert WikiCorpus into suitable form for Doc2Vec.

In [5]:
class TaggedWikiDocument(object):
    def __init__(self, wiki):
        self.wiki = wiki
        self.wiki.metadata = True
        self.count = 0
    def __iter__(self):
        for content, (page_id, title) in self.wiki.get_texts():
            if self.count < 10:
                print(page_id)
                print(title)
                print(content)
                self.count+=1

            yield TaggedDocument([c.decode("utf-8") for c in content], [title])

In [6]:
documents = TaggedWikiDocument(wiki)

In [7]:
count_doc = 0
for doc in documents:
    if count_doc < 10:
        print(doc)
        count_doc += 1
    else:
        break

12
Anarchism
[b'anarchism', b'is', b'political', b'philosophy', b'that', b'advocates', b'self', b'governed', b'societies', b'based', b'on', b'voluntary', b'institutions', b'these', b'are', b'often', b'described', b'as', b'stateless', b'societies', b'although', b'several', b'authors', b'have', b'defined', b'them', b'more', b'specifically', b'as', b'institutions', b'based', b'on', b'non', b'hierarchical', b'free', b'associations', b'anarchism', b'holds', b'the', b'state', b'to', b'be', b'undesirable', b'unnecessary', b'and', b'harmful', b'while', b'anti', b'statism', b'is', b'central', b'anarchism', b'generally', b'entails', b'opposing', b'authority', b'or', b'hierarchical', b'organisation', b'in', b'the', b'conduct', b'of', b'all', b'human', b'relations', b'including', b'but', b'not', b'limited', b'to', b'the', b'state', b'system', b'anarchism', b'is', b'usually', b'considered', b'radical', b'left', b'wing', b'ideology', b'and', b'much', b'of', b'anarchist', b'economics', b'and', b'anar

## Preprocessing
To set the same vocabulary size with original papar. We first calculate the optimal **min_count** parameter.

In [8]:
# pre = Doc2Vec(min_count=0)
# pre.scan_vocab(documents)

In [9]:
#for num in range(0, 20):
#    print('min_count: {}, size of vocab: '.format(num), pre.scale_vocab(min_count=num, dry_run=True)['memory']['vocab']/700)

In the original paper, they set the vocabulary size 915,715. It seems similar size of vocabulary if we set min_count = 19. (size of vocab = 898,725)

## Training the Doc2Vec Model
To train Doc2Vec model by several method, DBOW and DM, we define the list of models.

In [10]:
cores = multiprocessing.cpu_count()
print('Cores: ' + str(cores))

models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, size=200, window=8, min_count=19, iter=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=19, iter =10, workers=cores),
]

Cores: 8


In [11]:
models[0].build_vocab(documents)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

2017-05-29 19:40:32,768 : INFO : collecting all words and their counts
2017-05-29 19:40:33,075 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-29 19:41:20,201 : INFO : PROGRESS: at example #10000, processed 31159151 words (661208/s), 441596 word types, 10000 tags
2017-05-29 19:41:42,707 : INFO : finished iterating over Wikipedia corpus of 14901 documents with 46096422 positions (total 19857 articles, 46117421 positions before pruning articles shorter than 50 words)
2017-05-29 19:41:42,708 : INFO : collected 551375 word types and 14901 unique tags from a corpus of 14901 examples and 46096422 words
2017-05-29 19:41:42,709 : INFO : Loading a fresh vocabulary
2017-05-29 19:41:43,179 : INFO : min_count=19 retains 62534 unique words (11% of original 551375, drops 488841)
2017-05-29 19:41:43,180 : INFO : min_count=19 leaves 44748674 word corpus (97% of original 46096422, drops 1347748)
2017-05-29 19:41:43,340 : INFO : deleting the raw counts dictionary 

Doc2Vec(dbow+w,d200,n5,w8,mc19,s0.001,t8)
Doc2Vec(dm/m,d200,n5,w8,mc19,s0.001,t8)


Now we’re ready to train Doc2Vec of the English Wikipedia. 

In [12]:
for model in models:
    model.train(documents, total_examples=model.corpus_count, epochs=model.iter)

2017-05-29 19:41:45,588 : INFO : training model with 8 workers on 62534 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=8
2017-05-29 19:41:46,645 : INFO : PROGRESS: at 0.02% examples, 92835 words/s, in_qsize 7, out_qsize 0
2017-05-29 19:41:47,647 : INFO : PROGRESS: at 0.05% examples, 127810 words/s, in_qsize 16, out_qsize 1
2017-05-29 19:41:48,665 : INFO : PROGRESS: at 0.10% examples, 151163 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:41:49,668 : INFO : PROGRESS: at 0.15% examples, 163586 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:41:50,702 : INFO : PROGRESS: at 0.20% examples, 170053 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:41:51,719 : INFO : PROGRESS: at 0.26% examples, 173930 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:41:52,735 : INFO : PROGRESS: at 0.32% examples, 177738 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:41:53,739 : INFO : PROGRESS: at 0.41% examples, 178875 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:41:54,752 : IN

2017-05-29 19:43:07,617 : INFO : PROGRESS: at 4.49% examples, 190973 words/s, in_qsize 14, out_qsize 0
2017-05-29 19:43:08,628 : INFO : PROGRESS: at 4.55% examples, 191035 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:43:09,638 : INFO : PROGRESS: at 4.60% examples, 191228 words/s, in_qsize 14, out_qsize 0
2017-05-29 19:43:10,648 : INFO : PROGRESS: at 4.65% examples, 191069 words/s, in_qsize 14, out_qsize 0
2017-05-29 19:43:11,666 : INFO : PROGRESS: at 4.69% examples, 190963 words/s, in_qsize 14, out_qsize 1
2017-05-29 19:43:12,721 : INFO : PROGRESS: at 4.76% examples, 190899 words/s, in_qsize 11, out_qsize 0
2017-05-29 19:43:13,732 : INFO : PROGRESS: at 4.81% examples, 190999 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:43:14,794 : INFO : PROGRESS: at 4.88% examples, 191140 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:43:15,959 : INFO : PROGRESS: at 4.93% examples, 190909 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:43:16,985 : INFO : PROGRESS: at 4.98% examples, 191040 word

2017-05-29 19:44:30,149 : INFO : PROGRESS: at 9.08% examples, 192047 words/s, in_qsize 16, out_qsize 0
2017-05-29 19:44:31,258 : INFO : PROGRESS: at 9.15% examples, 191898 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:44:32,332 : INFO : PROGRESS: at 9.20% examples, 191930 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:44:33,340 : INFO : PROGRESS: at 9.27% examples, 191930 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:44:34,428 : INFO : PROGRESS: at 9.33% examples, 192039 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:44:35,443 : INFO : PROGRESS: at 9.37% examples, 192018 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:44:36,459 : INFO : PROGRESS: at 9.42% examples, 192087 words/s, in_qsize 14, out_qsize 1
2017-05-29 19:44:37,467 : INFO : PROGRESS: at 9.48% examples, 192026 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:44:38,517 : INFO : PROGRESS: at 9.55% examples, 192068 words/s, in_qsize 12, out_qsize 0
2017-05-29 19:44:39,563 : INFO : PROGRESS: at 9.61% examples, 192060 word

2017-05-29 19:45:49,934 : INFO : PROGRESS: at 13.50% examples, 191899 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:45:50,948 : INFO : PROGRESS: at 13.55% examples, 191921 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:45:51,981 : INFO : PROGRESS: at 13.61% examples, 191916 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:45:53,025 : INFO : PROGRESS: at 13.67% examples, 191933 words/s, in_qsize 13, out_qsize 2
2017-05-29 19:45:54,107 : INFO : PROGRESS: at 13.73% examples, 191971 words/s, in_qsize 13, out_qsize 0
2017-05-29 19:45:55,113 : INFO : PROGRESS: at 13.77% examples, 191926 words/s, in_qsize 14, out_qsize 0
2017-05-29 19:45:56,118 : INFO : PROGRESS: at 13.83% examples, 191924 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:45:57,216 : INFO : PROGRESS: at 13.89% examples, 191888 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:45:58,247 : INFO : PROGRESS: at 13.95% examples, 191921 words/s, in_qsize 14, out_qsize 0
2017-05-29 19:45:59,253 : INFO : PROGRESS: at 14.01% examples, 1

2017-05-29 19:47:11,179 : INFO : PROGRESS: at 17.90% examples, 192018 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:47:12,197 : INFO : PROGRESS: at 17.96% examples, 192069 words/s, in_qsize 12, out_qsize 0
2017-05-29 19:47:13,231 : INFO : PROGRESS: at 18.04% examples, 192062 words/s, in_qsize 11, out_qsize 0
2017-05-29 19:47:14,298 : INFO : PROGRESS: at 18.11% examples, 192058 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:47:15,354 : INFO : PROGRESS: at 18.20% examples, 192058 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:47:16,388 : INFO : PROGRESS: at 18.27% examples, 192093 words/s, in_qsize 13, out_qsize 0
2017-05-29 19:47:17,421 : INFO : PROGRESS: at 18.33% examples, 192036 words/s, in_qsize 15, out_qsize 2
2017-05-29 19:47:18,466 : INFO : PROGRESS: at 18.39% examples, 192070 words/s, in_qsize 12, out_qsize 0
2017-05-29 19:47:19,480 : INFO : PROGRESS: at 18.43% examples, 192027 words/s, in_qsize 15, out_qsize 0
2017-05-29 19:47:20,489 : INFO : PROGRESS: at 18.48% examples, 1

...

2017-05-29 20:22:31,327 : INFO : PROGRESS: at 72.65% examples, 396050 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:22:32,365 : INFO : PROGRESS: at 72.76% examples, 396034 words/s, in_qsize 2, out_qsize 0
2017-05-29 20:22:33,411 : INFO : PROGRESS: at 72.86% examples, 396008 words/s, in_qsize 0, out_qsize 2
2017-05-29 20:22:34,417 : INFO : PROGRESS: at 72.98% examples, 396051 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:22:35,454 : INFO : PROGRESS: at 73.10% examples, 396029 words/s, in_qsize 4, out_qsize 0
2017-05-29 20:22:36,462 : INFO : PROGRESS: at 73.22% examples, 396026 words/s, in_qsize 10, out_qsize 0
2017-05-29 20:22:37,475 : INFO : PROGRESS: at 73.37% examples, 396097 words/s, in_qsize 0, out_qsize 1
2017-05-29 20:22:38,481 : INFO : PROGRESS: at 73.49% examples, 396096 words/s, in_qsize 0, out_qsize 1
2017-05-29 20:22:39,481 : INFO : PROGRESS: at 73.60% examples, 396134 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:22:40,482 : INFO : PROGRESS: at 73.72% examples, 396150 wo

2017-05-29 20:23:50,731 : INFO : PROGRESS: at 81.64% examples, 395974 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:23:51,740 : INFO : PROGRESS: at 81.73% examples, 395952 words/s, in_qsize 10, out_qsize 0
2017-05-29 20:23:52,763 : INFO : PROGRESS: at 81.85% examples, 396001 words/s, in_qsize 6, out_qsize 0
2017-05-29 20:23:53,765 : INFO : PROGRESS: at 81.98% examples, 396009 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:23:54,814 : INFO : PROGRESS: at 82.08% examples, 396021 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:23:55,865 : INFO : PROGRESS: at 82.21% examples, 396007 words/s, in_qsize 1, out_qsize 2
2017-05-29 20:23:56,867 : INFO : PROGRESS: at 82.35% examples, 396042 words/s, in_qsize 5, out_qsize 0
2017-05-29 20:23:57,876 : INFO : PROGRESS: at 82.48% examples, 396050 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:23:58,877 : INFO : PROGRESS: at 82.59% examples, 396058 words/s, in_qsize 4, out_qsize 0
2017-05-29 20:23:59,887 : INFO : PROGRESS: at 82.71% examples, 396110 wo

2017-05-29 20:25:10,187 : INFO : PROGRESS: at 90.64% examples, 396095 words/s, in_qsize 2, out_qsize 1
2017-05-29 20:25:11,188 : INFO : PROGRESS: at 90.80% examples, 396094 words/s, in_qsize 2, out_qsize 0
2017-05-29 20:25:12,189 : INFO : PROGRESS: at 90.95% examples, 396106 words/s, in_qsize 0, out_qsize 1
2017-05-29 20:25:13,203 : INFO : PROGRESS: at 91.06% examples, 396080 words/s, in_qsize 1, out_qsize 0
2017-05-29 20:25:14,234 : INFO : PROGRESS: at 91.17% examples, 396067 words/s, in_qsize 7, out_qsize 0
2017-05-29 20:25:15,237 : INFO : PROGRESS: at 91.29% examples, 396137 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:25:16,237 : INFO : PROGRESS: at 91.38% examples, 396116 words/s, in_qsize 4, out_qsize 0
2017-05-29 20:25:17,329 : INFO : PROGRESS: at 91.52% examples, 396126 words/s, in_qsize 0, out_qsize 0
2017-05-29 20:25:18,347 : INFO : PROGRESS: at 91.60% examples, 396123 words/s, in_qsize 1, out_qsize 0
2017-05-29 20:25:19,359 : INFO : PROGRESS: at 91.71% examples, 396109 wor

2017-05-29 20:26:31,800 : INFO : PROGRESS: at 99.98% examples, 396368 words/s, in_qsize 0, out_qsize 1
2017-05-29 20:26:31,895 : INFO : finished iterating over Wikipedia corpus of 14901 documents with 46096422 positions (total 19857 articles, 46117421 positions before pruning articles shorter than 50 words)
2017-05-29 20:26:31,902 : INFO : worker thread finished; awaiting finish of 7 more threads
2017-05-29 20:26:31,904 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-05-29 20:26:31,904 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-05-29 20:26:31,905 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-05-29 20:26:31,906 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-29 20:26:31,906 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-29 20:26:31,907 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-29 20:26:31,919 : INFO : worker thread finishe

CPU times: user 32min 45s, sys: 15.9 s, total: 33min 1s
Wall time: 14min 40s


## Save the models ##

In [13]:
models[0].save('models/wikipedia_dbow_model.doc2vec')
models[1].save('models/wikipedia_dm_model.doc2vec')

2017-05-29 20:26:31,925 : INFO : saving Doc2Vec object under wikipedia_dbow_model.doc2vec, separately None
2017-05-29 20:26:31,927 : INFO : not storing attribute syn0norm
2017-05-29 20:26:31,927 : INFO : storing np array 'syn0' to wikipedia_dbow_model.doc2vec.wv.syn0.npy
2017-05-29 20:26:31,960 : INFO : not storing attribute cum_table
2017-05-29 20:26:31,961 : INFO : storing np array 'syn1neg' to wikipedia_dbow_model.doc2vec.syn1neg.npy
2017-05-29 20:26:32,284 : INFO : saved wikipedia_dbow_model.doc2vec
2017-05-29 20:26:32,285 : INFO : saving Doc2Vec object under wikipedia_dm_model.doc2vec, separately None
2017-05-29 20:26:32,286 : INFO : not storing attribute syn0norm
2017-05-29 20:26:32,286 : INFO : storing np array 'syn0' to wikipedia_dm_model.doc2vec.wv.syn0.npy
2017-05-29 20:26:32,317 : INFO : not storing attribute cum_table
2017-05-29 20:26:32,318 : INFO : storing np array 'syn1neg' to wikipedia_dm_model.doc2vec.syn1neg.npy
2017-05-29 20:26:32,640 : INFO : saved wikipedia_dm_mode

## Similarity interface

After that, let's test both models! DBOW model show the simillar results with the original paper. First, calculating cosine simillarity of "Machine learning" using Paragraph Vector. Word Vector and Document Vector are separately stored. We have to add .docvecs after model name to extract Document Vector from Doc2Vec Model.

In [14]:
for model in models:
    print(str(model))
    pprint(model.docvecs.most_similar(positive=["Anarchism"], topn=20))

2017-05-29 20:26:32,645 : INFO : precomputing L2-norms of doc weight vectors


Doc2Vec(dbow+w,d200,n5,w8,mc19,s0.001,t8)


TypeError: unorderable types: str() < int()

DBOW model interpret the word 'Machine Learning' as a part of Computer Science field, and DM model as Data Science related field.

Second, calculating cosine simillarity of "Lady Gaga" using Paragraph Vector.

In [ ]:
for model in models:
    print(str(model))
    pprint(model.docvecs.most_similar(positive=["Lady Gaga"], topn=10))

DBOW model reveal the similar singer in the U.S., and DM model understand that many of Lady Gaga's songs are similar with the word "Lady Gaga".

Third, calculating cosine simillarity of "Lady Gaga" - "American" + "Japanese" using Document vector and Word Vectors. "American" and "Japanese" are Word Vectors, not Paragraph Vectors. Word Vectors are already converted to lowercases by WikiCorpus.

In [ ]:
for model in models:
    print(str(model))
    vec = [model.docvecs["Lady Gaga"] - model["american"] + model["japanese"]]
    pprint([m for m in model.docvecs.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

As a result, DBOW model demonstrate the similar artists with Lady Gaga in Japan such as 'Perfume', which is the Most famous Idol in Japan. On the other hand, DM model results don't include the Japanese aritsts in top 10 simillar documents. It's almost same with no vector calculated results.

This results demonstrate that DBOW employed in the original paper is outstanding for calculating the similarity between Document Vector and Word Vector.